<a href="https://colab.research.google.com/github/seanpaulbradley/nmatchcv/blob/main/Copy_of_THINGSVISION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# THINGSvision
This is the PyTorch version, you can find a TensorFlow example [here](https://colab.research.google.com/github/ViCCo-Group/THINGSvision/blob/master/doc/tensorflow.ipynb)

## Settings

### Install thingsvision and dependencies

In [ ]:
!pip install --upgrade thingsvision
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
!pip install ipywidgets

In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
from thingsvision import get_extractor
from thingsvision.utils.storing import save_features
from thingsvision.utils.data import ImageDataset, DataLoader
from thingsvision.core.extraction import center_features

from google.colab import drive
from typing import Any, Dict, List, Optional, Union

### Image and feature directories

Specify both `path/to/images` (input directory) and `path/to/features` (output directory) on your Google Drive.
The image directory is expected to contain images that are saved similarly to `/dog/img_1.png` or `/cat/img_1.jpg`.

In [ ]:
image_dir = 'path/to/images'  # path/to/images in GDrive
output_dir = 'path/to/features' # path/to/output  in GDrive

Mount Google Drive

In [ ]:
mounted_dir = '/thingsvision'
drive.mount(mounted_dir, force_remount=True)

In [ ]:
full_image_path = os.path.join(mounted_dir, 'MyDrive', image_dir)
full_output_path = os.path.join(mounted_dir, 'MyDrive', output_dir)

### Helper functions to extract features

In [ ]:
def extract_features(
    extractor: Any,
    module_name: str,
    image_path: str,
    out_path: str,
    batch_size: int,
    flatten_activations: bool,
    apply_center_crop: bool,
    class_names: Optional[List[str]]=None,
    file_names: Optional[List[str]]=None,
) -> Union[np.ndarray, torch.Tensor]:
    """Extract features for a single layer."""
    dataset = ImageDataset(
        root=image_path,
        out_path=out_path,
        backend=extractor.get_backend(),
        transforms=extractor.get_transformations(apply_center_crop=apply_center_crop, resize_dim=256, crop_dim=224),
        class_names=class_names,
        file_names=file_names,
    )
    batches = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        backend=extractor.get_backend(),
        )
    features = extractor.extract_features(
        batches=batches,
        module_name=module_name,
        flatten_acts=flatten_activations,
        output_type="ndarray", # or "tensor" (only applicable to PyTorch models)
    )
    return features


def extract_all_layers(
    model_name: str,
    extractor: Any,
    image_path: str,
    out_path: str,
    batch_size: int,
    flatten_activations: bool,
    apply_center_crop: bool,
    layer: Any=nn.Linear,
    file_format: str = "npy",
    class_names: Optional[List[str]]=None,
    file_names: Optional[List[str]]=None,
) -> Dict[str, Union[np.ndarray, torch.Tensor]]:
    """Extract features for all selected layers and save them to disk."""
    features_per_layer = {}
    for l, (module_name, module) in enumerate(extractor.model.named_modules(), start=1):
        if isinstance(module, layer):
            # extract features for layer "module_name"
            features = extract_features(
                extractor=extractor,
                module_name=module_name,
                image_path=image_path,
                out_path=out_path,
                batch_size=batch_size,
                flatten_activations=flatten_activations,
                apply_center_crop=apply_center_crop,
                class_names=class_names,
                file_names=file_names,
            )
            # replace with e.g., [f'conv_{l:02d}'] or [f'fc_{l:02d}']
            features_per_layer[f'layer_{l:02d}'] = features
            # save features to disk
            save_features(features, out_path=f'{out_path}/features_{model_name}_{module_name}', file_format=file_format)
    return features_per_layer

### Variables

In [ ]:
pretrained = True # use pretrained model weights
model_path = None # if pretrained = False (i.e., randomly initialized weights) set path to model weights
batch_size = 32 # use a power of two (this can be any size, depending on the number of images for which you aim to extract features)
apply_center_crop = True # center crop images (set to False, if you don't want to center-crop images)
flatten_activations = True # whether or not features (e.g., of Conv layers) should be flattened
class_names = None  # optional list of class names for class dataset
file_names = None # optional list of file names according to which features should be sorted
file_format = "npy" # format with which to save features to disk (can be set to "mat", "txt", "npy", "hdf5")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Select `model` and `layer` for which you want to extract image features. If you want to extract features from a `torchvision` model, use the model naming defined [here](https://pytorch.org/vision/stable/models.html) (e.g., `vgg16` if you want to use VGG-16). If you are uncertain about the naming and enumeration of the layers, use `extractor.show_model()` to see how specific layers are called. This command will show the architecture of the model.

### Example 1: VGG-16 with batch norm (pretrained on ImageNet)

Note that it is crucial to set a model's `source`. VGG16 implementations exist in different libraries and therefore (pretrained) weights can be downloaded from different sources. One such source is `torchvision` from which we will download VGG16.

In [ ]:
# load model
model_name = 'vgg16_bn'
# specify model source
# we use torchvision here (https://pytorch.org/vision/stable/models.html)
source = 'torchvision'
# initialize the extractor
extractor = get_extractor(
    model_name=model_name,
    pretrained=pretrained,
    model_path=model_path,
    device=device,
    source=source
)

In [ ]:
## select layer

# NOTE: uncomment the line below, if you are uncertain about layer naming in PyTorch
# extractor.show_model()
module_name = 'features.23'

#### Feature extraction single layer

In [ ]:
# extract features for a single layer (e.g., penultimate)
features = extract_features(
    extractor=extractor,
    module_name=module_name,
    image_path=full_image_path,
    out_path=full_output_path,
    batch_size=batch_size,
    flatten_activations=flatten_activations,
    apply_center_crop=apply_center_crop,
    class_names=class_names,
    file_names=file_names,
)

# apply centering (not necessary, but may be desirable, depending on the analysis)
features = center_features(features)

# save features to disk
save_features(features, out_path=f'{full_output_path}/features_{model_name}_{module_name}', file_format=file_format)

#### Feature extraction all convolutional or fully-connected layers

In [ ]:
# extract features for all convolutional layers (i.e., Conv2d) and save them to disk
layer = nn.Conv2d
features_conv_layers = extract_all_layers(
    model_name=model_name,
    extractor=extractor,
    image_path=full_image_path,
    out_path=full_output_path,
    batch_size=batch_size,
    flatten_activations=flatten_activations,
    apply_center_crop=apply_center_crop,
    layer=layer,
    file_format=file_format,
    class_names=class_names,
    file_names=file_names,
)

In [ ]:
# extract features for all fully-connected layers (i.e., Linear) and save them to disk
layer = nn.Linear
features_fc_layers = extract_all_layers(
    model_name=model_name,
    extractor=extractor,
    image_path=full_image_path,
    out_path=full_output_path,
    batch_size=batch_size,
    flatten_activations=flatten_activations,
    apply_center_crop=apply_center_crop,
    layer=layer,
    file_format=file_format,
    class_names=class_names,
    file_names=file_names,
)

### Example 2: VGG-16 (pretrained on Ecoset)

In [ ]:
## load model
model_name = 'VGG16_ecoset'
# specifiy model source
# the model's source here is custom
source = 'custom'
extractor = get_extractor(
    model_name=model_name,
    pretrained=pretrained,
    model_path=model_path,
    device=device,
    source=source,
)

In [ ]:
## select layer

# NOTE: uncomment the line below, if you are uncertain about layer naming in PyTorch and look at architecture
# extractor.show_model()
module_name = 'features.23'

#### Feature extraction single layer

In [ ]:
# extract features for a single layer (e.g., penultimate)
vgg_ecoset_features = extract_features(
    extractor=extractor,
    module_name=module_name,
    image_path=full_image_path,
    out_path=full_output_path,
    batch_size=batch_size,
    flatten_activations=flatten_activations,
    apply_center_crop=apply_center_crop,
    class_names=class_names,
    file_names=file_names,
)

# apply centering (not necessary, but may be desirable, depending on the analysis)
vgg_ecoset_features = center_features(vgg_ecoset_features)

# save features to disk
save_features(vgg_ecoset_features, out_path=f'{full_output_path}/features_{model_name}_{module_name}', file_format=file_format)

#### Feature extraction all convolutional or fully-connected layers

In [ ]:
# extract features for all convolutional layers (i.e., Conv2d) and save them to disk
layer = nn.Conv2d
features_conv_layers = extract_all_layers(
    model_name=model_name,
    extractor=extractor,
    image_path=full_image_path,
    out_path=full_output_path,
    batch_size=batch_size,
    flatten_activations=flatten_activations,
    apply_center_crop=apply_center_crop,
    layer=layer,
    file_format=file_format,
    class_names=class_names,
    file_names=file_names,
)

In [ ]:
# extract features for all fully-connected layers (i.e., Linear) and save them to disk
layer = nn.Linear
features_fc_layers = extract_all_layers(
    model_name=model_name,
    extractor=extractor,
    image_path=full_image_path,
    out_path=full_output_path,
    batch_size=batch_size,
    flatten_activations=flatten_activations,
    apply_center_crop=apply_center_crop,
    layer=layer,
    file_format=file_format,
    class_names=class_names,
    file_names=file_names,
)

### Example 3: CLIP (multimodal pretraining)

In [ ]:
# load model
model_name = 'clip'
variant = 'ViT-B/32'
# specifiy model source
# the model's source here is custom
source = 'custom'
extractor = get_extractor(
    model_name=model_name,
    pretrained=pretrained,
    model_path=model_path,
    device=device,
    source=source,
    model_parameters={'variant': variant}
)

In [ ]:
## select layer

# NOTE: uncomment the line below, if you are uncertain about layer naming of CLIP
# extractor.show_model()
module_name = 'visual' # image features

#### Feature extraction single layer

In [ ]:
# extract features
clip_features = extract_features(
    extractor=extractor,
    module_name=module_name,
    image_path=full_image_path,
    out_path=full_output_path,
    batch_size=batch_size,
    flatten_activations=flatten_activations,
    apply_center_crop=apply_center_crop,
    class_names=class_names,
    file_names=file_names,
)

# apply centering (not necessary, but may be desirable, depending on the analysis)
clip_features = center_features(clip_features)

# save features to disk
save_features(clip_features, out_path=f'{full_output_path}/features_{model_name}_{module_name}', file_format=file_format)

### Representational Similarity Analysis (RSA)

In [ ]:
from thingsvision.core.rsa import compute_rdm, plot_rdm

In [ ]:
# compute representational dissimilarity matrix (RDM) for CLIP features
rdm = compute_rdm(clip_features, method='correlation')

In [ ]:
# plot rdm
plot_rdm(
        full_output_path,
        clip_features,
        method='correlation',
        format='.png', # '.jpg'
        colormap='cividis',
        show_plot=True,
)

### Centered Kernel Alignment (CKA)

In [ ]:
from thingsvision.core.cka import CKA

In [ ]:
assert vgg_ecoset_features.shape[0] == clip_features.shape[0]
m = clip_features.shape[0]
cka = CKA(m=m, kernel='linear')
rho = cka.compare(X=vgg_ecoset_features, Y=clip_features)